# $M$/$M$/$1$ queue simulation

The following notebook imports a C function into Python using the ctypes library which allows to simulate a trajectory of a $M$/$M$/$1$ queue.

In [1]:
import numpy as np
import ctypes
import plotly.express as px
from plotly_resampler import register_plotly_resampler
from random import randint
from sys import maxsize

SO_PATH = "c/main.so"
#register_plotly_resampler(mode='auto')

In [2]:
simulation_lib = ctypes.CDLL(SO_PATH)

simulation_lib.simulation.restype = ctypes.POINTER(ctypes.POINTER(ctypes.c_double))
simulation_lib.simulation.argtypes = (ctypes.c_double, ctypes.c_double, ctypes.c_double,
                                      ctypes.c_uint, ctypes.POINTER(ctypes.c_size_t))

In [3]:
class MM1Queue:
    def __init__(self, T: int, λ: float, μ: float, seed = -1) -> None:
        self.T = T
        self.λ = λ
        self.μ = μ
        self.ρ = float(self.λ)/float(self.μ)
        if(seed == -1):
            self.seed = randint(1, maxsize)
        else:
            self.seed = seed
        self.results = None
        self._simulate()
        
    def _simulate(self):
        T = ctypes.c_double(self.T)
        λ = ctypes.c_double(self.λ)
        μ = ctypes.c_double(self.μ)
        seed = ctypes.c_uint(self.seed)
        size = ctypes.c_size_t()
        results_ptr = simulation_lib.simulation(T, λ, μ, seed, ctypes.byref(size))
        results = results_ptr[:size.value]
        result_arr = np.zeros((size.value, 2))
        for i in range(size.value):
            result_arr[i][0] = results[i][0]
            result_arr[i][1] = results[i][1]
        for i in range(size.value):
            simulation_lib.free(results[i])
        simulation_lib.free(results_ptr)
        self.results = result_arr
    
    def trajectory_plot(self):
        if(self.results is not None):
            fig = px.line(x=self.results[:,0], y=self.results[:,1], line_shape='hv', title="M/M/1 queue" )
            fig.update_layout(
                xaxis_title="t", yaxis_title="Number of clients in system"
            )
            fig.show()
        else:
            print("Simulation not executed!")
    
    def server_utilization_plot(self):
        if(self.results is not None):
            m = self.results
            t0 = np.where(m[:,1] == 0, m[:,0], -1)
            t1 = np.zeros(len(t0))
            t1[:-1] = np.where(m[1:,1] == 1, m[1:,0], -1)
            diff = np.where((t0 != -1) & (t1 != -1), t1-t0, 0)
            v = 1 - np.cumsum(diff[1:])/m[1:,0]
            fig = px.line(x=m[1:-1,0], y=v[:-1], title="M/M/1 queue" )
            fig.add_hline(y=self.ρ,  line_color="red")
            fig.update_layout(
                            xaxis_title="t", yaxis_title="% Server Utilization"
            )
            fig.show()
        else:
            print("Simulation not executed!")

    def number_in_system_plot(self):
        if(self.results is not None):
            m = self.results
            t0 = m[:,0]
            t1 = np.zeros(len(t0))
            t1[:-1] = t0[1:]
            diff = t1-t0
            v = np.cumsum(m[1:,1]*diff[1:])/m[1:,0]
            fig = px.line(x=m[1:-1,0], y=v[:-1], title="M/M/1 queue" )
            fig.add_hline(y=self.ρ/(1-self.ρ),  line_color="red")
            fig.update_layout(
                            xaxis_title="t", yaxis_title="Expected number in system"
            )
            fig.show()
        else:
            print("Simulation not executed!")

In [5]:
queue = MM1Queue(1000, 2, 5, 123)
queue.trajectory_plot()
queue.server_utilization_plot()
queue.number_in_system_plot()